# **BILBIOTHEQUES**

In [1]:
import json

import pandas as pd
import numpy as np

import chess
import random

from Fct.fct_preprocess import *
from Fct.fct_eval import *

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Conv2D, Flatten, Dropout, Dense, BatchNormalization, MaxPooling2D
from tensorflow.keras.optimizers import Adam


import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'pandas'

---

# **1. EXTRACTION DES DONNEES BRUTES**

Extraction des données depuis le fichier JSONL d'origine qui contient 137 000 000 positions différentes, chacune avec plusieurs évaluations stockfish . Nous limitons alors la taille de notre base de donnée a **500 000 positions**.

In [8]:
file_path = 'data\lichess_db_eval.jsonl'
data_raw = extract_data(file_path, max_rows=5000)
data_raw.head(5)

,fen,knodes,depth,cp,mate,line
0,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,200973,39,58.0,NaN,f7g7 e6e2 h8d8 e2d2 b7b5 c4e6 g7f6 e6b3 a6a5 a2a3
1,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,62.0,NaN,f7g7 e6e2 b7b5 c4b3 h8d8 e2d2 a6a5 a2a3 g7f6 d1e1
2,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,151.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4a2 c6d7 e6e7 f7g6 e1f2
3,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,59730,31,64.0,NaN,f7g7 e6e2 g7g6 d1c2 h8d8 e2d2 g6f6 a2a3 b7b5 c4a2
4,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,59730,31,134.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4b3 a5a4 b3a2 c6d7 e6h6


La base de donnée est composée de **2 330 098 évaluations différentes** et **5 variables** : 

| Variable            | Description                                                                                                        |
|-----------------|--------------------------------------------------------------------------------------------------------------------|
| **knodes**      | Nombre de milliers de nœuds analysés par stockfish pendant la recherche de la position.                   |
| **depth**       | Profondeur de la recherche, indiquant le nombre de coups (ou niveaux) explorés par le moteur.                     |
| **cp**          | Évaluation en centipions, représentant un avantage matériel en fonction du côté actif (positif pour Blancs, négatif pour Noirs). |
| **mate**        | Évaluation de mat. Si une valeur est donnée, elle indique le nombre de coups restants avant que le mat soit atteint. |
| **line**        | Ligne (parfois appelée variante) désigne une séquence de coups qui découle d'une position donnée. Elle représente une possible continuation du jeu |

---

# **2. NETTOYAGE DES DONNEES**

Stockfish utilie différents parametre dans son algorythme, notamment le **nombre de noeuf (knodes)** et le la **profondeur (depth)**. En fonction de ces différents parametres, stockfish calcul le **centipions (cp)** et **l'évaluation du mate (mate)**.  

Après ces évaluations, stockfish peut alors parfois prédire des coups différents pour une même position, on appelle cela des **variantes**. Ces coups sont différents mais ont un impacte similaire ou quasi-similaire sur l'avantage qu'il donne a celui qui le joue (en terme d'evaluation de la position). 

Certains joueurs vont être plus a l'aise dans une variante plûtot qu'une autre en fonction de leurs styles de jeu ou de leurs connaissances, mais pour une IA cela n'as pas d'importance ! Toutefois, notre source de donnée nous propose plusieurs analyse de stockfish pour chaques position, apportant son lot de variante.

Nous pouvons alors utiliser 2 méthodes pour garder la meilleur variante.

### **Méthode N°1 : Analyse des Parametres et de l'Evaluation**
   1. **Profondeur de recherche (depth)** : Plus la profondeur est grande, plus l'analyse est précise. Priorisations des évaluations avec la profondeur maximale.

   2. **Centipions (cp) ou mat (mate)** : Si une évaluation donne un **mate**, elle est prioritaire, car un mat forcé est absolu. En absence de **mate**, choix d'évaluation avec la valeur **cp** la plus élevée pour le joueur actif

   3. **Nombre de nœuds analysés (knodes)** : Si plusieurs évaluations ont la même profondeur, celle ayant exploré le plus grand nombre de nœuds est théoriquement plus fiable.

   4. **En cas d'égalité parfaite**: En cas d’égalité sur les autres critères, utilisez une évaluation arbitraire.

### **Méthode N°2 : Coup le Plus Populaire** 
blablablabla


- Création d'une variable `best_move_m1`, qui extrait le prochain coup a jouer, c'est a dire le premier coup de la ligne.

In [9]:
data_V1 = extract_next_move(data_raw)
data_V1.head(3)

,fen,knodes,depth,cp,mate,line,best_move_m1
0,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,200973,39,58.0,NaN,f7g7 e6e2 h8d8 e2d2 b7b5 c4e6 g7f6 e6b3 a6a5 a2a3,f7g7
1,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,62.0,NaN,f7g7 e6e2 b7b5 c4b3 h8d8 e2d2 a6a5 a2a3 g7f6 d1e1,f7g7
2,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,151.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4a2 c6d7 e6e7 f7g6 e1f2,h8d8


- Création d'une variable `best_move_m2` avec le coup le plus populaire par position en suivant la Méthode N°2

In [10]:
data_V2 = most_popular_predict_V1(data_V1)
data_V2.head(3)

,fen,knodes,depth,cp,mate,line,best_move_m1,best_move_m2
0,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,200973,39,58.0,NaN,f7g7 e6e2 h8d8 e2d2 b7b5 c4e6 g7f6 e6b3 a6a5 a2a3,f7g7,f7g7
1,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,62.0,NaN,f7g7 e6e2 b7b5 c4b3 h8d8 e2d2 a6a5 a2a3 g7f6 d1e1,f7g7,f7g7
2,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,151.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4a2 c6d7 e6e7 f7g6 e1f2,h8d8,f7g7


- Filtrage des lignes pour ne garder qu'une evaluation par position selon la méthode N°1.

In [11]:
data_V3 = filter_best_predict(data_V2)
data_V3.head(3)

,fen,knodes,depth,cp,mate,line,best_move_m1,best_move_m2
0,1K6/1P4p1/2n4k/p6p/8/8/8/8 w - -,159161,29,NaN,-12.0,b8a8 a5a4 b7b8b c6b8 a8b8 a4a3 b8c8 a3a2 c8d7 ...,b8a8,b8c8
1,1K6/1P4p1/7k/p3n2p/8/8/8/8 b - -,186187,25,460.0,NaN,e5c6 b8c7 c6b4 c7b6 b4d5 b6c6 d5b4 c6b5 h6g5 b...,e5c6,e5c6
2,1K6/2Pk4/b7/5ppp/8/p5P1/4PP1P/B7 w - -,1472,39,0.0,NaN,e2e3 a3a2 a1c3 a6c8 f2f3 g5g4 f3g4 h5g4 c3d4 c8a6,e2e3,e2e3


- Suppression des variables inutiles. Les variables `knodes`, `depth`, `cp`, `mate` et `line` ne sont plus utile pour la suite du projet

In [12]:
data_cleaned = drop_usuless_columns(data_V3)
data_cleaned.head(5)

,fen,best_move_m1,best_move_m2
0,1K6/1P4p1/2n4k/p6p/8/8/8/8 w - -,b8a8,b8c8
1,1K6/1P4p1/7k/p3n2p/8/8/8/8 b - -,e5c6,e5c6
2,1K6/2Pk4/b7/5ppp/8/p5P1/4PP1P/B7 w - -,e2e3,e2e3
3,1Kb2n2/2P1k3/2B5/4Bppp/8/p5P1/4PP1P/8 b - -,f8d7,c8e6
4,1Kb5/2PBk3/8/4Bppp/8/p5P1/4PP1P/8 b - -,c8d7,e7d7


- Sauvegarde des données netoyées en CSV

In [13]:
data_cleaned.to_csv('data/data_cleaned.csv')

---

# **3. ENTRAINEMENT DES MODELES**

### **Transformation Finale des Données** :

blablablabla

- Chargement des Données

In [2]:
data = pd.read_csv('data/data_cleaned.csv' )

X = data['fen']
y = data['best_move_m1']

- Traduction des données sous une forme interpretable par le reseau de neurone.
  - **Position (Fen)** -> Matrice Numpy
  - **Coup a joué (UCI)** -> Encodage de chaque coup distincts (sous forme d'entier)

In [3]:
X = X.apply(fen_to_matrix)
X = np.array(X.tolist())

# Encode les mouvements et conversion en catégories
y, move_to_int = encode_moves(y)
y = to_categorical(y, num_classes=len(move_to_int))


# Sauvegarde de move_to_int dans un fichier JSON
with open('Models/move_int_dico.json', 'w') as file:
    json.dump(move_to_int, file)


KeyboardInterrupt



### **Modèle N°1** :

- Definition du Modèle

In [15]:
model_1 = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(13, 8, 8)),
    BatchNormalization(),  # Ajout de BatchNormalization pour stabiliser l'entraînement
    MaxPooling2D(pool_size=(2, 2)),  # Réduction de la dimension spatiale
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dropout(0.4),  # Dropout réduit à 40% ici pour limiter le surapprentissage
    Dense(256, activation='relu'),
    Dropout(0.3),  # Dropout progressif pour éviter un réseau trop complexe
    Dense(len(move_to_int), activation='softmax')  # Couche de sortie
])

# Optimiseur avec un learning rate ajusté
model_1.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

model_1.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 11, 6, 64)      │         4,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 11, 6, 64)      │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 5, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 3, 1, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 3, 1, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 1, 0, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 0)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 0)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1927)           │       495,239 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 574,791 (2.19 MB)

 Trainable params: 574,407 (2.19 MB)

 Non-trainable params: 384 (1.50 KB)

- Entrainement du Modèle

In [ ]:
# model_1.fit(X, y, epochs=25, validation_split=0.1, batch_size=64)
## Sauvegarde
#model_1.save("Models/Modele_1_TF_25EPOCHS.keras")

### **Modèle N°2** :

Pareil mais avec un entrainement different : epoch 50 et batch_size 126

In [ ]:
# model_2.fit(X, y, epochs=50, validation_split=0.1, batch_size=126)
## Sauvegarde
#model_2.save("Models/Modele_2_TF_50EPOCHS_1M.keras")

# **4. EVALUATION DES MODELES**

In [3]:
# Chargement des différents models

model_1 = load_model("Models\Modele_1_TF_25EPOCHS.keras")
model_2 = load_model("Models\Modele_2_TF_50EPOCHS_1M.keras.keras")

# Chargement du dictionnaire de traduction coup / integer pour l'interpretation des modèles
with open('Models/move_int_dico.json', 'r') as file:
    move_int = json.load(file)
move_int_dico = {v: k for k, v in move_int.items()}

ValueError: File not found: filepath=Models\Modele_1_TF_25EPOCHS.keras. Please ensure the file is an accessible `.keras` zip file.

In [3]:
play_game(model_1,"random",move_int_dico,print_game = True)

Début de la partie !
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 396ms/step
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . P . .
. . . . . . . .
P P P P P . P P
R N B Q K B N R 

r n b q k b n r
p p p p . p p p
. . . . p . . .
. . . . . . . .
. . . . . P . .
. . . . . . . .
P P P P P . P P
R N B Q K B N R 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
r n b q k b n r
p p p p . p p p
. . . . p . . .
. . . . . . . .
. . . . . P . .
. . N . . . . .
P P P P P . P P
R . B Q K B N R 

r n b q k b . r
p p p p . p p p
. . . . p n . .
. . . . . . . .
. . . . . P . .
. . N . . . . .
P P P P P . P P
R . B Q K B N R 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
r n b q k b . r
p p p p . p p p
. . . . p n . .
. . . . . . . .
. . . . . P . .
. . N . . . . .
P P P P P . P P
. R B Q K B N R 

r n b q k b . r
p p p p . p p p
. . . . p . . .
. . . . . . . .
. . . . n P . .
. . N 

'1/2-1/2'

A FAIRE -> METTRE EN PLACE LA NUL PAR REPETITION OU A 100 COUPS !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
# fonction cassé a réparé :

def play_multiple_games(ia1, ia2, n, model1=None, model2=None, move_int_dico=None):
    """
    Fait jouer plusieurs parties entre deux IA et retourne les résultats cumulés.

    Arguments:
    - ia1, ia2 : Fonctions des IA pour les joueurs.
    - n : Nombre de parties à jouer.
    - model1, model2 : Modèles TensorFlow (facultatifs).
    - move_int_dico : Dictionnaire pour les prédictions (facultatif).

    Retourne:
    - Résumé des résultats : victoires IA1, victoires IA2, égalités.
    """
    ia1_wins = 0
    ia2_wins = 0
    draws = 0

    for i in range(n):
        print(f"\nPartie {i+1}:")
        moves, result = play_game_auto(ia1, ia2, model1, model2, move_int_dico)
        
        if result == "1-0":
            ia1_wins += 1
        elif result == "0-1":
            ia2_wins += 1
        else:
            draws += 1

    # Résultats
    print("\nRésultats finaux:")
    print(f"Victoires IA1 : {ia1_wins}")
    print(f"Victoires IA2 : {ia2_wins}")
    print(f"Égalités : {draws}")

    return {"IA1 Wins": ia1_wins, "IA2 Wins": ia2_wins, "Draws": draws}
